In [ ]:
var Constants = {
  speechBonus: 0.1,
  listenBonus: 0.1,
}

function choice(l=[]){
  return l[Math.floor(Math.random() * l.length)];
}

function mean(l=[]){
  var sum = 0.0;
  for(var i=0; i<l.length; i++){
  	sum += l[i];
  }
  return sum/l.length;
}

In [ ]:
function Golem(){
  this.health = Math.random();
  this.lapses = 0.0;
}

Golem.prototype.speak = function(){
  this.health += Math.random() * Constants.speechBonus;
  return Math.random();
}

Golem.prototype.age = function(){
  this.lapses += 1.0;
  return this.lapses >= 1000.0;
}

Golem.prototype.listenTo = function(speech){
  var impact = Math.random() * speech * Constants.listenBonus;
  this.health *= impact
}

Golem.prototype

In [ ]:
function Setting(n){
  this.golems = [];
  for(var i=0; i<n; i++){
    this.golems.push(new Golem());
  }
  
  console.log(`Setting created, <br>Number of Golems: ${this.golems.length} <br>Mean Health: ${mean(this.golems.map(g => g.health))}.`)
}

Setting.prototype.historize = function(fact){
  if(this.history === undefined){
  	this.history = {};
  }
  
  for(var attribute of Object.keys(fact)){
  	if(this.history[attribute] === undefined){
      this.history[attribute] = [];
    }
    
  	this.history[attribute].push(fact[attribute]);
  }
}

Setting.prototype.round = function(){
  var speaker = choice(this.golems);
  var listener = choice(this.golems);

  var speech = speaker.speak();
  listener.listenTo(speech);

  if(listener.health > 0.075){
	  this.golems.push(new Golem());
  }
  
  // each golem ages
  this.golems.forEach((golem, i) => {
  	if(golem.age()){
    	this.golems.splice(i, 1);
    }
	})
  var meanHealth = mean(this.golems.map(g => g.health));
  var meanAge = mean(this.golems.map(g => g.lapses));
  
  this.historize({
  	'round'      : this.history.round.length, 
    'mean health': meanHealth,
  	'count'      : this.golems.length,
    'mean age'   : meanAge
  });
  
	return mean(this.golems.map(g => g.health));
}

Setting.prototype

In [ ]:
var html = `<h2>Mean Health over Round</h2><div id="chart" style="display: block;"></div>`
html

In [ ]:
var html2 = `<h2>Mean Age over Round</h2><div id="chart-2" style="display: block;"></div>`
html2

In [ ]:
var html3 = `<h2>Count over Round</h2><div id="chart-3" style="display: block;"></div>`
html3

# Animation Code

In [ ]:
var setting = new Setting(10)

var chart = document.querySelector('es-notebook').shadowRoot.querySelectorAll('notebook-cell')[3].shadowRoot.querySelector('#chart');

vegaEmbed(chart, {
  $schema: 'https://vega.github.io/schema/vega-lite/v5.json',
  data: {name: 'table'},
  width: 580,
  mark: 'line',
  encoding: {
    x: {field: 'round', type: 'ordinal', scale: {zero: false}},
    y: {field: 'mean health', type: 'quantitative'}
  }
}).then(function(res) {
  function newGenerator(){
	  var counter = -1;
    var previousY = [0];
  
    return function(){
    	counter++;
      
    	var newVals = previousY.map(function(v, c){
        return {
        	'round': counter,
          'mean health': setting.round()
        }
      });
      
      previousY = newVals.map(function(v){
        return v['mean health'];
      });
      
      return newVals;
    };
  }

  var valueGenerator = newGenerator();
  var minimumX = -20;
  window.setInterval(function () {
    minimumX++;
    var changeSet = vega
      .changeset()
      .insert(valueGenerator())
      .remove(function (t) {
        return t['round'] < minimumX;
      });
    res.view.change('table', changeSet).run();
  }, 100);
});

setting.historize({round: 0})

In [ ]:
var chart = document.querySelector('es-notebook').shadowRoot.querySelectorAll('notebook-cell')[4].shadowRoot.querySelector('#chart-2');

vegaEmbed(chart, {
  $schema: 'https://vega.github.io/schema/vega-lite/v5.json',
  data: {name: 'table'},
  width: 600,
  mark: 'line',
  encoding: {
    x: {field: 'round', type: 'ordinal', scale: {zero: false}},
    y: {field: 'mean age', type: 'quantitative'}
  }
}).then(function(res) {
  function newGenerator(){
	  var counter = -1;
    var previousY = [0];
  
    return function(){
    	counter++;
      setting.round()
      
    	var newVals = previousY.map(function(v, c){
        return {
        	'round': counter,
          'mean age': setting.history['mean age'][setting.history['mean age'].length - 1]
        }
      });
      
      previousY = newVals.map(function(v){
        return v['mean age'];
      });
      
      return newVals;
    };
  }

  var valueGenerator = newGenerator();
  var minimumX = -20;
  window.setInterval(function () {
    minimumX++;
    var changeSet = vega
      .changeset()
      .insert(valueGenerator())
      .remove(function (t) {
        return t['round'] < minimumX;
      });
    res.view.change('table', changeSet).run();
  }, 100);
});

setting.historize({round: 0})

In [ ]:
var chart = document.querySelector('es-notebook').shadowRoot.querySelectorAll('notebook-cell')[5].shadowRoot.querySelector('#chart-3');

vegaEmbed(chart, {
  $schema: 'https://vega.github.io/schema/vega-lite/v5.json',
  data: {name: 'table'},
  width: 600,
  mark: 'line',
  encoding: {
    x: {field: 'round', type: 'ordinal', scale: {zero: false}},
    y: {field: 'count', type: 'quantitative'}
  }
}).then(function(res) {
  function newGenerator(){
	  var counter = -1;
    var previousY = [0];
  
    return function(){
    	counter++;
      setting.round()
      
    	var newVals = previousY.map(function(v, c){
        return {
        	'round': counter,
          'count': setting.history['count'][setting.history['count'].length - 1]
        }
      });
      
      previousY = newVals.map(function(v){
        return v['count'];
      });
      
      return newVals;
    };
  }

  var valueGenerator = newGenerator();
  var minimumX = -20;
  window.setInterval(function () {
    minimumX++;
    var changeSet = vega
      .changeset()
      .insert(valueGenerator())
      .remove(function (t) {
        return t['round'] < minimumX;
      });
    res.view.change('table', changeSet).run();
  }, 100);
});

setting.historize({round: 0})

In [ ]:
setting.golems.push(new Golem())